In [1]:
# Import helpful Python packages
import pandas as pd
import datetime
from IPython.display import HTML
pd.set_option('display.max_rows', 500)

# Load the data
Teams = pd.read_csv('/kaggle/input/meta-kaggle/Teams.csv',low_memory=False)
Competitions = pd.read_csv('/kaggle/input/meta-kaggle/Competitions.csv',low_memory=False)
ForumTopics = pd.read_csv('/kaggle/input/meta-kaggle/ForumTopics.csv',low_memory=False)
ForumMessages = pd.read_csv('/kaggle/input/meta-kaggle/ForumMessages.csv',low_memory=False)

# Rename a few columns
Teams = Teams.rename(columns={'Id': 'Id_Teams'})
ForumTopics = ForumTopics.rename(columns={'Id': 'Id_ForumTopics',
                                         'Title': 'Title of Writeup',
                                         'CreationDate': 'Date of Writeup'})
ForumMessages = ForumMessages.rename(columns={'Id': 'Id_ForumMessages',
                                              'Message': 'Writeup'})
Competitions = Competitions.rename(columns={'Id': 'Id_Competitions',
                                           'Title': 'Title of Competition',
                                           'EnabledDate': 'Competition Launch Date'})

# Organize everything
df = Teams.merge(right=ForumTopics, how='inner', left_on='WriteUpForumTopicId', right_on='Id_ForumTopics')
df = df.merge(right=ForumMessages, how='inner', left_on='Id_ForumTopics', right_on='ForumTopicId')
df = df.merge(right=Competitions, how='inner', left_on='ForumId', right_on='ForumId')
df = df[df['FirstForumMessageId'] == df['Id_ForumMessages']]
df = df[df['HostSegmentTitle'].isin(['Featured','Research'])]

# Add in URLs
df['Id_Competitions'] = df['Id_Competitions'].astype(str)
df['Competition URL'] = 'https://www.kaggle.com/c/'+df['Id_Competitions']
df['Id_ForumTopics'] = df['Id_ForumTopics'].astype(str)
df['Writeup URL'] = 'https://www.kaggle.com/c/'+df['Id_Competitions']+'/discussion/'+df['Id_ForumTopics']

# Final cleanup
df = df[['Competition Launch Date',
         'Title of Competition',
         'Competition URL']].drop_duplicates()#,
#          'Date of Writeup',
#          'Title of Writeup',
#          'Writeup',
#          'Writeup URL'

# Save everything to a single CSV file
todays_date = f'kaggle_competitions_{datetime.datetime.now().strftime("%H%M_%m%d%Y")}'
df.to_csv('/kaggle/working/'+todays_date+'.csv',index=False)

# Make the links clickable and then preview the table
print('# of entries: ',df['Title of Competition'].count())
HTML(df[::-1].to_html(render_links=True, escape=False))

# of entries:  302


,Competition Launch Date,Title of Competition,Competition URL
46676,10/04/2022 15:13:27,G2Net Detecting Continuous Gravitational Waves,https://www.kaggle.com/c/37077
46520,09/21/2022 21:55:10,Novozymes Enzyme Stability Prediction,https://www.kaggle.com/c/37190
46151,08/30/2022 17:23:07,Feedback Prize - English Language Learning,https://www.kaggle.com/c/38321
45953,08/15/2022 20:49:22,Open Problems - Multimodal Single-Cell Integration,https://www.kaggle.com/c/38128
45912,08/11/2022 21:33:45,AI Village Capture the Flag @ DEFCON,https://www.kaggle.com/c/37381
45767,07/29/2022 21:49:47,DFL - Bundesliga Data Shootout,https://www.kaggle.com/c/37244
45563,07/28/2022 17:49:51,RSNA 2022 Cervical Spine Fracture Detection,https://www.kaggle.com/c/36363
45418,07/11/2022 17:09:06,Google Universal Image Embedding,https://www.kaggle.com/c/36414
45280,07/06/2022 20:34:00,Mayo Clinic - STRIP AI,https://www.kaggle.com/c/37333
45192,06/22/2022 18:34:11,HuBMAP + HPA - Hacking the Human Body,https://www.kaggle.com/c/34547
